In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_dir = os.environ['FINN_ROOT'] + "/notebooks"
model_file = model_dir + "/auto_mai_subj6_ready.onnx"

out_dir = "output3"

#Delete previous run results if exist
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
    print("Previous run results deleted!")


cfg = build.DataflowBuildConfig(
    output_dir=out_dir,
    # mvau_wwidth_max=36,
    target_fps=150,
    synth_clk_period_ns=10.0,
    # You can keep board="ZCU104" if you like, but the key is fpga_part:
    board="ZCU104",
    fpga_part="xczu7ev-ffvc1156-2-e",          # <-- explicit part resolves the error
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    save_intermediate_models=True,              # optional but helpful
)

Previous run results deleted!


In [2]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from /home/dan/Projects/fpga-eeg/extern/finn/notebooks/auto_mai_subj6_ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_dan
Final outputs will be generated in output2
Build log is at output2/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_out_of_context_synthesis [16/19]
Running step: step_synthesize

0

In [3]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_dir = os.environ['FINN_ROOT'] + "/notebooks/output2/intermediate_models"
model_file = model_dir + "/step_out_of_context_synthesis.onnx"

out_dir = "output2"

cfg = build.DataflowBuildConfig(
    output_dir=out_dir,
    start_step="step_synthesize_bitfile",
    # mvau_wwidth_max=36,
    target_fps=150,
    synth_clk_period_ns=10.0,
    # You can keep board="ZCU104" if you like, but the key is fpga_part:
    board="ZCU104",
    fpga_part="xczu7ev-ffvc1156-2-e",          # <-- explicit part resolves the error
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    save_intermediate_models=True,              # optional but helpful
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from intermediate checkpointoutput2/intermediate_models/step_out_of_context_synthesis.onnx
Intermediate outputs will be generated in /tmp/finn_dev_dan
Final outputs will be generated in output2
Build log is at output2/build_dataflow.log
Running step: step_synthesize_bitfile [1/3]
> /home/dan/Projects/fpga-eeg/extern/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py(269)apply()
    267         bitfile_name = vivado_pynq_proj_dir + "/finn_zynq_link.runs/impl_1/top_wrapper.bit"
    268         if not os.path.isfile(bitfile_name):
--> 269             raise Exception(
    270                 "Synthesis failed, no bitfile found. Check logs under %s" % vivado_pynq_proj_dir
    271             )



Traceback (most recent call last):
  File "/home/dan/Projects/fpga-eeg/extern/finn/src/finn/builder/build_dataflow.py", line 158, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/home/dan/Projects/fpga-eeg/extern/finn/src/finn/builder/build_dataflow_steps.py", line 796, in step_synthesize_bitfile
    model = model.transform(
  File "/home/dan/Projects/fpga-eeg/extern/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/home/dan/Projects/fpga-eeg/extern/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py", line 352, in apply
    model = model.transform(MakeZYNQProject(self.platform, enable_debug=self.enable_debug))
  File "/home/dan/Projects/fpga-eeg/extern/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/home/dan/Projects/fpga-eeg/